<a href="https://colab.research.google.com/github/allanbatista/fasttext-ecommerce.github.io/blob/gh-pages/notebooks/FastText_E_Commerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://storage.googleapis.com/black-magic-us-west1/fasttext/2020-07-16T00-00-00/model.bin

--2020-09-19 12:01:15--  https://storage.googleapis.com/black-magic-us-west1/fasttext/2020-07-16T00-00-00/model.bin
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.126.128, 74.125.70.128, 64.233.191.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7704734077 (7.2G) [application/octet-stream]
Saving to: ‘model.bin’

model.bin           100%[===================>]   7.17G  30.8MB/s    in 1m 51s  

2020-09-19 12:03:07 (65.9 MB/s) - ‘model.bin’ saved [7704734077/7704734077]



In [2]:
!pip install fasttext --quiet

     |████████████████████████████████| 71kB 2.1MB/s 


In [3]:
import fasttext

ft = fasttext.load_model('model.bin')

In [7]:
import re
import numpy as np
 
re_html = re.compile(r"<[^>]*>")
re_especial = re.compile(r'^[\-\.\,]+|\s[\-\.\,]+|[\-\.\,]+\s|[\-\.\,]+$')
re_chars = re.compile(r'[^a-z0-9àáâãçéêíóôõúü+\-\"\s\.\,]')
re_lines = re.compile(r'\n')
re_spaces = re.compile(r'\s+')
 
tokens_norm = dict(
   zip(
       'æ,œ,á,è,ì,ò,ù,ä,ë,ï,ö,ü,ÿ,â,ê,î,ô,û,å,ø,Ø,ñ'.split(","),
       'ae,oe,a,e,i,o,u,a,e,i,o,u,y,a,e,i,o,u,a,o,O,n'.split(",")
   )
)
 
 
def _accent2latin(text):
   result = []
 
   for char in text.lower():
       newchar = tokens_norm.get(char)
 
       if newchar:
           result.append(newchar)
       else:
           result.append(char)
 
   return "".join(result)
 
 
def clear(text):
   text = str(text).lower()
   text = _accent2latin(text)
   text = re_html.sub(' ', text)
   text = re_chars.sub(' ', text)
   text = re_lines.sub(' ', text)
   text = re_especial.sub(' ', text)
   text = re_spaces.sub(' ', text)
   return text.strip()

def text_to_vector(text):
    return np.array([ft.get_word_vector(word) for word in clear(text).split(" ")])

In [9]:
sequence = text_to_vector('Smartphone - Samsung Galaxy / S10e 128GB - Dual Chip, Android 9.0 Tela 5,8" | Octa-Core 4G, Câmera, 12MP + 16MP, - Preto')
sequence.shape

(18, 128)

In [10]:
sequence

array([[ 4.3844166e-01, -4.7509857e-02,  9.6057311e-02, ...,
         4.0147254e-01,  5.7129258e-01,  5.1002932e-01],
       [ 4.6748745e-01,  1.4027068e-01,  8.6354554e-01, ...,
         6.4235687e-01, -1.6558748e-01,  6.7117769e-01],
       [ 3.6011356e-01,  2.7058411e-01,  1.0783412e-01, ...,
         1.1131119e+00,  3.9792199e-02,  3.4689578e-01],
       ...,
       [ 8.3547008e-01,  2.8060443e+00,  2.2126739e+00, ...,
         1.6515121e+00, -4.4876766e+00,  1.0340667e+00],
       [ 7.0913267e-01, -7.6971769e-01,  2.6940376e-01, ...,
        -7.8384399e-02,  4.7494173e-01,  6.9902754e-01],
       [ 1.5348157e-01, -9.4336078e-02,  3.4438714e-01, ...,
         2.7751854e-01, -3.7175959e-03,  3.1282184e-01]], dtype=float32)